In [1]:
import pandas as pd
import owlready2 as owlr 
#https://pythonhosted.org/Owlready/
import rdflib
from collections import Counter
import uuid

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
raw_world = owlr.World(filename='raw_world.sqlite3')
rawdms = raw_world.get_ontology("datamodels.owl").load()
clean_world = owlr.World(filename='clean_world.sqlite3')
cleandms = clean_world.get_ontology("datamodels.owl").load()
onto = rawdms

In [61]:
g = rdflib.Graph()

In [3]:
onto_dataproperties = list(onto.data_properties())
onto_individuals = list (onto.individuals())
individuals = [(i, i.is_a.first()) for i in onto_individuals + onto_dataproperties]
serializations = [i[0] for i in individuals if i[1] == onto.Serialization]

for s in serializations:
    print(s)
    mappings = [m for m in s.SerializationContainsMapping]
    for m in mappings:
        print ("\t", m, m.is_a)

datamodels.DomainModelClassEntityRecord
	 datamodels.AttributeMapping [datamodels.Mapping]
	 datamodels.ClassMapping [datamodels.Mapping]
	 datamodels.DomainMapping [datamodels.Mapping]
	 datamodels.ModelMapping [datamodels.Mapping]
	 datamodels.hasAttributeMapping [datamodels.Mapping]
	 datamodels.hasClassMapping [datamodels.Mapping]
	 datamodels.hasDataTypeMapping [datamodels.Mapping]
	 datamodels.hasModelMapping [datamodels.Mapping]
	 datamodels.hasModelTypeMapping [datamodels.Mapping]


In [4]:
with onto:
    owlr.sync_reasoner(raw_world)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmprxz3zm37
* Owlready2 * HermiT took 0.9770960807800293 seconds
* Owlready * Equivalenting: datamodels.PartialIdentifier datamodels.r_PartialIdentifier
* Owlready * Equivalenting: datamodels.r_PartialIdentifier datamodels.PartialIdentifier
* Owlready * Equivalenting: datamodels.Identifier datamodels.r_Identifier
* Owlready * Equivalenting: datamodels.r_Identifier datamodels.Identifier
* Owlready * Reparenting datamodels.r_PartialIdentifier: {owl.ObjectProperty} => {datamodels.Attribute}
* Owlready * Reparenting datamodels.r_Identifier: {owl.ObjectProperty} => {datamodels.Attribute}
* Owlready * Reparenting datamodels.RelationshipFeaturesInModel: {owl.ObjectProperty} => {datamodels

In [5]:
mapped_things = [(c,c.MappingKey,"class") for c in onto.classes() if len(c.MappingKey) > 0] + \
                [(p, p.MappingKey,"data") for p in onto.data_properties() if len(p.MappingKey) > 0] + \
                [(p, p.MappingKey,"object") for p in onto.object_properties() if len(p.MappingKey) > 0] 
mappable_things = {v[0]:k for k,v,t in mapped_things}
mappable_things_ext = {v[0]:(k,t) for k,v,t in mapped_things}
assert all([v==1 for v in Counter([b for c in mapped_things for b in c[1]]).values()])
#del mapped_things
#mappable_things
mappable_things_ext

{'Attribute': (datamodels.Attribute, 'class'),
 'Class': (datamodels.Class, 'class'),
 'Context': (datamodels.Context, 'class'),
 'DataModel': (datamodels.DataModel, 'class'),
 'Mapping': (datamodels.Mapping, 'class'),
 'Domain': (datamodels.ModelDomain, 'class'),
 'Relationship': (datamodels.Relationship, 'class'),
 'Serialization': (datamodels.Serialization, 'class'),
 'hasDataType': (datamodels.DataType, 'data'),
 'hasModelType': (datamodels.ModelType, 'data'),
 'hasModelType_obsolete': (datamodels.ModelTypeCode, 'data'),
 'hasAttribute': (datamodels.ClassHasAttribute, 'object'),
 'isAttributeParent': (datamodels.IdentityAttributeParentClass, 'object'),
 'isClassParent': (datamodels.IdentityClassParentModel, 'object'),
 'isRelationshipParent': (datamodels.IdentityRelationshipParentModel,
  'object'),
 'hasClass': (datamodels.ModelContainsClass, 'object'),
 'hasModel': (datamodels.ModelDomainContainsModel, 'object')}

In [6]:
data_row = { "Business Domain" : "Core", 
             "Model" : "Principal Interactions", 
             "ModelType" : "Conceptual", 
             "Entity" : "Person", 
             "Attribute" : "PersonID", 
             "DataType" : "string"}

In [7]:
def instantiate_thing(label, tclass, space, test_for_existing=True):
    # Check to see if this object of the same class contains the same label
    # if so, then return the first item of that name
    if test_for_existing:
        matches = [i for i in space.search(type=tclass) if label in i.label]
        if len(matches)>0:
            return matches[0]
    # Otherwise, generate a unique ID, create the item in the space and assign this label
    guid = uuid.uuid4().hex
    with space:
        i = tclass(guid, namespace=space)
    i.label=label
    return i
    

In [8]:
def instantiate_property(subject_key, subject_domain, predicate, object_key, object_domain, space):
    # Get classes for domains and ranges
    sd_list = [c for c in list(space.classes()) if c==subject_domain]
    od_list = [c for c in list(space.classes()) if c==object_domain]
    
    # Get instances of classes that match provided keys
    if subject_key is None:
        c_s = [s for l in sd_list for s in l.instances()]
    else:
        c_s = [s for s in sd_list for s in l.instances() if subject_key in s.label]
        
    if object_key is None:
        c_o = [o for l in od_list for o in l.instances()]
    else:
        print ("labels", [(o.label, object_key in o.label, object_key) for o in od_list])
        c_o = [o for l in od_list for o in l.instances()if object_key in o.label]
    
    print(od_list, object_key)
    
    print(predicate)
    predicate.python_name="pp"
    print(c_s[0], predicate)
    if len(c_o)==0:
        print("property_data", predicate, object_key, od_list, predicate.is_a)
        # Sometimes fails for data_properties
        # prop_relation = c_s[0].pp.append(object_key)
        #prop_relation = space._add_obj_triple_spo(c_s[0].storid, predicate.storid, object_key)
        try:
            prop_relation=predicate[c_s[0]]=[object_key]
        except AttributeError as e:
            print(e)
            prop_relation=predicate[c_s[0]]=[object_key]
        print("success", prop_relation)
        print("value", predicate[c_s[0]], c_s[0])
    else:
        if predicate.is_functional_for(c_o[0]):
            prop_relation = c_s[0].pp=c_o[0]
        else:
            prop_relation = c_s[0].pp=[c_o[0]]
    predicate.python_name = ""
    return c_s, predicate, c_o, prop_relation

In [9]:
#ont = cleandms
#ont = onto
#q = instantiate_thing("test", onto.Class, cleandms, True)
#q = instantiate_thing("test 2", onto.Class, onto, True)

#for i in ont.Class.instances():
#    print (i.iri, i.label)

In [10]:
#for i in onto.Class.instances():
#    print (i.iri, i.label)

In [11]:
s_objects = []
p_objects = []
for s in serializations:
    print(s)
    mappings = [m for m in s.SerializationContainsMapping]
    serials = {}
    for m in mappings:
        s_key = m.SerializationLabel.first()
        i_key = m.MappingLabel.first()

        if s_key not in serials.keys():
            serials[s_key]=[]

        print ("\t", m, s_key)
            
        if m.is_a.first() == onto.EntityMapping:
            print("\t\tEntity:{e}".format(e=m))
            o_range = m.MappingRange.first()
            property_d = { "type" : "entity", 
                           "mapping_name" : i_key, 
                           "property" : "rdf:type:", 
                           "range" :    mappable_things[o_range], 
                           "property_type" : mappable_things_ext[i_key],
                           "subject_label" : data_row[s_key]}
#            serials[s_key].append((data_row[s_key], "rdf:type", mappable_things[o_range]))
            serials[s_key].append(property_d)
    
        elif m.is_a.first() == onto.PropertyMapping:
            print("\t\tProperty:{e}".format(e=m))
            s_domain = m.MappingDomain.first()
            o_range = m.MappingRange.first()
            property_d = { "type" : "property", 
                           "mapping_name" : i_key, 
                           "domain" :   mappable_things[s_domain], 
                           "property" : mappable_things[i_key], 
                           "property_type" : mappable_things_ext[i_key],
                           "range" :    mappable_things.get(o_range,o_range), 
                           "range_search_value" : data_row[s_key]}
            serials[s_key].append(property_d)
            
        else:
            print(m.is_a)
            print(s_key)
            assert False
            
print()

# Instantiate temporary versions of all the in-row objects
for v in serials.values():
    for u in v:
        print(u)
        if u['type'] == "entity":#u[1]=="rdf:type":
            #with onto:
                #q = instantiate_thing("test", onto.Class, cleandms, True)
            s_objects.append(instantiate_thing(u['subject_label'], u['range'], onto, True))
                #s_objects.append(instantiate_thing(u[0], u[2], onto, True))
                #u[2](u[0], namespace=onto) # Create instances in the ontology
            print(u)

print()


datamodels.DomainModelClassEntityRecord
	 datamodels.AttributeMapping Attribute
		Entity:datamodels.AttributeMapping
	 datamodels.ClassMapping Entity
		Entity:datamodels.ClassMapping
	 datamodels.DomainMapping Business Domain
		Entity:datamodels.DomainMapping
	 datamodels.ModelMapping Model
		Entity:datamodels.ModelMapping
	 datamodels.hasAttributeMapping Attribute
		Property:datamodels.hasAttributeMapping
	 datamodels.hasClassMapping Entity
		Property:datamodels.hasClassMapping
	 datamodels.hasDataTypeMapping DataType
		Property:datamodels.hasDataTypeMapping
	 datamodels.hasModelMapping Model
		Property:datamodels.hasModelMapping
	 datamodels.hasModelTypeMapping ModelType
		Property:datamodels.hasModelTypeMapping

{'type': 'entity', 'mapping_name': 'Attribute', 'property': 'rdf:type:', 'range': datamodels.Attribute, 'property_type': (datamodels.Attribute, 'class'), 'subject_label': 'PersonID'}
15 13 14
{'type': 'entity', 'mapping_name': 'Attribute', 'property': 'rdf:type:', 'range': d

In [12]:

# Create property links for all identified in-row objects
for v in serials.values():
    for u in v:
        if u['type'] == "property":

            p_objects.append(u)
            instantiate_property(None, u['domain'], u['property'], u["range_search_value"], u['range'], onto)


labels [(['Attribute'], False, 'PersonID')]
[datamodels.Attribute] PersonID
datamodels.ClassHasAttribute
datamodels.1e877a9d54b947ae9d416ade2ff1e80d datamodels.ClassHasAttribute
13 13 14
labels [(['Class'], False, 'Person')]
[datamodels.Class] Person
datamodels.ModelContainsClass
datamodels.1d7149959fbe473ab47cb2db8437cac2 datamodels.ModelContainsClass
13 13 14
labels [([], False, 'Principal Interactions')]
[datamodels.DataModel] Principal Interactions
datamodels.ModelDomainContainsModel
datamodels.a5b7171cb1094246808c88ae279bba43 datamodels.ModelDomainContainsModel
13 13 14
labels []
[] string
datamodels.DataType
datamodels.67c3edfc375a44ea86edae5e746f399d datamodels.DataType
property_data datamodels.DataType string [] [owl.DatatypeProperty]
14 13 14
success ['string']
value ['string'] datamodels.67c3edfc375a44ea86edae5e746f399d
labels []
[] Conceptual
datamodels.ModelType
datamodels.1d7149959fbe473ab47cb2db8437cac2 datamodels.ModelType
property_data datamodels.ModelType Conceptual []

In [16]:
dm=[i for i in onto.individuals() if 'Principal Interactions' in i.label][0]
#dm.get_properties()
print (dm, dm.is_a)
for p in dm.INDIRECT_get_properties():
    print(p,p[dm])

datamodels.1d7149959fbe473ab47cb2db8437cac2 [datamodels.DataModel]
datamodels.ModelIsMemberOfModelDomain [datamodels.a5b7171cb1094246808c88ae279bba43]
rdf-schema.label ['Principal Interactions']
datamodels.MappingKey []


In [60]:
list(raw_world.sparql("""SELECT ?s ?o 
WHERE { ?s rdfs:label ?o }
BIND "Person" as ?o """))

ParsingError: ('Error at BIND', 41)

In [19]:
p_objects[-1]

{'type': 'property',
 'mapping_name': 'hasModelType',
 'domain': datamodels.DataModel,
 'property': datamodels.ModelType,
 'property_type': (datamodels.ModelType, 'data'),
 'range': 'ModelType',
 'range_search_value': 'Conceptual'}

In [21]:
p_objects[-2]

{'type': 'property',
 'mapping_name': 'hasDataType',
 'domain': datamodels.Attribute,
 'property': datamodels.DataType,
 'property_type': (datamodels.DataType, 'data'),
 'range': 'DataType',
 'range_search_value': 'string'}

In [32]:
for p in p_objects:
    print(p['property'][attrib])#dm)

[]
[]
[]
['string']
[]


In [14]:
assert False

AssertionError: 

In [ ]:
onto.ModelTypeCode._python_name

In [ ]:
onto.ModelTypeCode.is_a[0].iri=="http://www.w3.org/2002/07/owl#DatatypeProperty"

In [ ]:
dir(onto.ModelTypeCode)

In [ ]:
[(i, i.is_a, i.label) for i in list(onto.individuals())]

In [31]:
attrib=[i for i in onto.individuals() if 'PersonID' in i.label][0]
attrib.get_properties()


{datamodels.DataType, rdf-schema.label, datamodels.AttributeIsMemberOfClass}

In [ ]:
for p in attrib.INDIRECT_get_properties():
    print(p,p[attrib])

In [25]:
dm=[i for i in onto.individuals() if 'Principal Interactions' in i.label][0]
#dm.get_properties()
print (dm, dm.is_a)
for p in dm.INDIRECT_get_properties():
    print(p,p[dm])

datamodels.1d7149959fbe473ab47cb2db8437cac2 [datamodels.DataModel]
datamodels.ModelIsMemberOfModelDomain [datamodels.a5b7171cb1094246808c88ae279bba43]
rdf-schema.label ['Principal Interactions']
datamodels.MappingKey []


In [ ]:
for e,p in enumerate(onto.data_properties()):
    print(p[dm])

In [ ]:
assert False

In [ ]:
for e,p in enumerate(onto.data_properties()):
    p[dm]=["Test"]
    print(e,p==[p for p in list(onto.data_properties()) ][8])
print("Done")
[p for p in list(onto.data_properties()) ][8][dm]=["Tester"]

In [ ]:
dir(onto)

In [ ]:
onto.Class.instances(), onto.Attribute.instances(), onto.DataModel.instances(), onto.ModelDomain.instances()

In [ ]:
onto.Attribute.instances()[0].get_properties()

In [ ]:
list(onto.Attribute.instances()[0]._get_instance_possible_relations())

In [ ]:
dir(onto.DataModel.instances()[0])

In [ ]:
for p in onto.Attribute.instances()[0].get_properties():
    print(p, p.python_name, p[onto.DataModel.instances()[0]])
        

In [ ]:
for p in onto.DataModel.instances()[0].get_properties():
    print(p, p.python_name, p[onto.DataModel.instances()[0]])
        

In [ ]:
onto.ModelTypeCode[onto.DataModel.instances()[0]]=['Test']

In [ ]:
onto.ModelTypeCode.python_name = "mtc"
#onto.DataModel.instances()[0].mtc=["Conceptual"]

In [ ]:
onto.DataModel.instances()[0].ModelTypeCode

In [ ]:
with onto:
    owlr.sync_reasoner(raw_world)

In [ ]:
[c for c in list(onto.classes()) if c==p_objects[0]['domain']][0].instances()

In [ ]:
p_objects[0]['property'].is_functional_for(p_objects[0]['range'])

In [ ]:
instantiate_property(None, p_objects[0]['domain'], p_objects[0]['property'], p_objects[0]['range_search_value'], p_objects[0]['range'], onto)

In [ ]:
[(s,s.is_a, s.label) for s in s_objects]

In [ ]:
[(c,c.iri, c.is_a, c.label) for c in onto.Class.instances()]

In [ ]:
assert False

In [ ]:
serials['Entity']

In [ ]:
with onto:
    owlr.sync_reasoner()

In [ ]:
onto.get_properties()

In [ ]:
list(onto.inconsistent_classes())

In [ ]:
for c in list(onto.classes()):
    print (c, c.instances())